# 클로저 : 프로그래밍의 즐거움 Chapter 6. 지연과 불변성

## Younggun Na (amazingguni@gmail.com)

### Overview

1. 함수 구문의 종류
2. 클로저
3. 재귀적으로 생각하기
4. A* 경로 찾기

함수형 프로그래밍의 핵심 중 람다 계산이라 불리는 방식이 있는데 클로저 함수는 람다 계산을 고수하는 일급 클래스

- 다른 함수에 인자로 전달하거나 결과로 리턴할 수 있다.
- 함수 합성, 부분적 평가, 재귀, 렉시컬 클로저, 순수 함수, 함수 제약, 고차 함수, 일급 클래스 함수

### 7.1 함수 구문의 종류

대부분의 클로저 복합 타입들은 그 요소에 대한 함수로 사용 가능

In [1]:
([:a :b] 0)

:a

요런 것을 함수로서 호출하면 유용하게 쓸 수 있다.

`[]`는 백터 `#{}`는 셋이다.

In [3]:
(println (map [:chthon :phthor :heowulf :grendel] #{0 3}))

(:chthon :grendel)


컬렉션이 함수로 기능할 수 있고 함수도 데이터로 기능할 수 있다. 

이게 바로 일급 클래스 함수로 불리는 개념

### 7.1.1 일급 클래스 함수

클로저는 개발 문제에 대해 함수를 데이터에 적용하는 관점으로 인식한다.

- 자바에서는 프로그램 내의 모든 행위들에 대해서 클래스에 덧붙여진 형태로 모델링되어야 한다는 관점

**일급 클래스의 정의**

컴퓨터 프로그래밍 언어 디자인에서, 특정 언어의 일급 객체 (일급 값, 일급 엔티티, 혹은 일급 시민)이라 함은 일반적으로 다른 객체들에 적용 가능한 연산을 모두 지원하는 객체를 가리킨다. 함수에 매개변수로 넘기기, 변수에 대입하기와 같은 연산들이 여기서 말하는 일반적인 연산의 예에 해당한다.

**조건**

- 일급 클래스는 필요에 의해 생성될 수 있다.
- 일급 클래스는 데이터 구조 내에 저장될 수 있다.
- 일급 클래스는 함수의 인자로 전달될 수 있다.
- 일급 클래스는 함수의 결과 값으로 리턴될 수 있다.

#### 필요에 의한 함수 생성: 합성 사용하기

클로저의 연산의 주 단위는 함수라는 것은 명백하다.

또한 함수를 새로 생성하거나 다른 함수들과 합성하는 것도 가능하다.

> comp 함수는 뒤에 나오는 함수들을 합성해준다.


In [3]:
(def fifth (comp first rest rest rest rest))
(fifth [1 2 3 4 5 6 7])

5

In [4]:
(first (rest (rest (rest (rest [1 2 3 4 5 6 7])))))

5

n번째 요소를 리턴하는 함수를 만들면

In [5]:
(defn fnth [n]
    (apply comp
           (cons first
                 (take (dec n) (repeat rest)))))

((fnth 5) '[a b c d e])

e

In [7]:
(println (take 3 (repeat rest)))

(#function[clojure.core/rest--4343] #function[clojure.core/rest--4343] #function[clojure.core/rest--4343])


이런식으로 합성의 관점에서 생각하면 유용하게 새 함수를 생성할 수 있다.

> 만일 한 줄에 여러 괄호가 등장한다면, 대부분 함수를 리턴하려는 의도로 생각해도 좋다.

In [9]:
(println (map (comp keyword #(.toLowerCase %) name) `(a B C)))

(:a :b :c)


#### 필요에 의한 함수 생성: 부분 함수 사용하기

partial 함수를 이용해서 추가적인 인자를 적용하는 새로운 함수를 구성할 수 있다.


In [13]:
(def won2dolor (partial * 1200))

#'user/won2dolor

In [14]:
(won2dolor 10)

12000

In [15]:
(won2dolor 10 3 4)

144000

#### complement로 불리언 값 변경하기

참 또는 거짓을 리턴하는 함수를 받아서 반대 값을 리턴한다.

In [16]:
(let [truthiness (fn [v] v)]
    [((complement truthiness) true)
     ((complement truthiness) 42)
     ((complement truthiness) false)
     ((complement truthiness) nil)])

[false false true true]

In [17]:
((complement even?) 2)

false

#### 함수를 데이터로 사용하기

일급 클래스 함수는 데이터 그 자체이기 때문에 어떤 컨테이너에도 저장할 수 있다.

- 데이터 조각, 로컬, 참조, 컬렉션 등 모든

함수를 데이터로 다룰 때 유용한 메소드의 예로 테스트 프레임워크인 clojure.test를 보자

> 함수의 인자 앞에 오는 map은 함수의 메타 데이타이다.

> (interpose sep coll) interpose 는 coll의 element들 사이에 sep를 추가한 lazy seq를 리턴해준다.
Returns a stateful transducer when no collection is provided.

In [18]:
(defn join
  {:test (fn []
            (assert
              (= (join "," [1 2 3]) "1,3,3")))}
  [sep s]
  (apply str (interpose sep s)))

#'user/join

In [19]:
(use '[clojure.test :as t])
(t/run-tests)


Testing user

ERROR in (join) (:3)
Uncaught exception, not in assertion.
expected: nil
  actual: java.lang.AssertionError: Assert failed: (= (join "," [1 2 3]) "1,3,3")
 at user$fn__72.invokeStatic (:3)
    user/fn (:2)
    clojure.test$test_var$fn__7983.invoke (test.clj:716)
    clojure.test$test_var.invokeStatic (test.clj:716)
    clojure.test$test_var.invoke (test.clj:707)
    clojure.test$test_vars$fn__8005$fn__8010.invoke (test.clj:734)
    clojure.test$default_fixture.invokeStatic (test.clj:686)
    clojure.test$default_fixture.invoke (test.clj:682)
    clojure.test$test_vars$fn__8005.invoke (test.clj:734)
    clojure.test$default_fixture.invokeStatic (test.clj:686)
    clojure.test$default_fixture.invoke (test.clj:682)
    clojure.test$test_vars.invokeStatic (test.clj:730)
    clojure.test$test_all_vars.invokeStatic (test.clj:736)
    clojure.test$test_ns.invokeStatic (test.clj:757)
    clojure.test$test_ns.invoke (test.clj:742)
    clojure.core$map$fn__4785.invoke (core.clj:26

{:test 1, :pass 0, :fail 0, :error 1, :type :summary}